### Specify the simulation directory and the target file number to analyze

In [16]:
#simulation_directory = "/home/novac/Desktop/qv3d-automation/simulation-7.03e14"
simulation_directory = '/home/novac/Desktop/my_projects/qv3d-automation/data/densScan-febe_acc'
file_number = 6
fontsize = 12

In [17]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.integrate import simpson
from scipy.integrate import simps
from scipy.constants import c, pi, e, m_e, epsilon_0

In [18]:
# IMPORTANT - Set these parameters 
only_driver = 0 # if witness included set this to 0
simulation_directory = '/home/novac/Desktop/my_projects/qv3d-automation/simulation-densScan'
parent_directory = '/home/novac/Desktop/my_projects/qv3d-automation'
synchrotron_dotpy = os.path.join(parent_directory, 'pyScripts', 'synchrotron.py')
arr = np.array([5e14, 6e14, 7e14, 8e14, 9e14, 1e15, 2e15, 3e15, 4e15, 5e15, 6e15, 
                 7e15, 8e15, 9e15, 1e16, 2e16, 3e16, 4e16, 5e16, 6e16, 7e16, 8e16, 9e16, 1e17])
simulation_is_running = 0

file_number = 6
fontsize = 12

In [19]:
if file_number < 10:
    file_name = 'v3d_synchrotron_0000' + str(file_number) + '.h5'
elif file_number < 100:
    file_name = 'v3d_synchrotron_000' + str(file_number) + '.h5'
elif file_number < 1000:
    file_name = 'v3d_synchrotron_00' + str(file_number) + '.h5'
else:
    print('ERROR: check the file_number.')

# Get the parent directory
#parent_directory = os.path.abspath(os.path.join(simulation_directory, os.pardir))
parent_directory = '/home/novac/Desktop/my_projects/qv3d-automation'
file_path = os.path.join(simulation_directory, 'density-7e14', file_name)
synchrotron_dotpy = os.path.join(parent_directory, 'pyScripts', 'synchrotron.py')


In [20]:
synch = !python {synchrotron_dotpy} {file_path}

In [21]:
synch[0]

'Traceback (most recent call last):'

In [6]:
my_dict = eval(synch[0])

# Convert NumPy arrays
for key, value in my_dict.items():
    if isinstance(value, np.ndarray):
        my_dict[key] = value.tolist()
        
keys = my_dict.keys()
print(keys) 

SyntaxError: invalid syntax (<string>, line 1)

In [ ]:
criticalEnergys = my_dict['criticalEnergy']
Nphotons_Energy = my_dict['Nphoton_Energy']
Nphotons_Theta  = my_dict['Nphoton_Theta']
Nphotons_Phi    = my_dict['Nphoton_Phi']
Energy          = my_dict['Energy']
Theta           = my_dict['Theta']
Phi             = my_dict['Phi']
phase           = my_dict['phase']

In [ ]:
def plot_spectrum(Nphotons_Energy, Energy):
    fig,ax = plt.subplots(figsize=(8,5))
    fig.tight_layout(pad=4.) #set the spacing between subplots

    #n0 = 7e14*1e6  # /m3 maximum plasma density
    #wp = np.sqrt(n0*e**2/epsilon_0/m_e)
    #dist = phase/wp * c

    ax.loglog(Energy,Nphotons_Energy,c='b') # photon number ~ beta oscillations * number of electrons

    ax.set_xlabel('Photon energy [eV]', fontsize=fontsize)
    ax.set_ylabel('Photons / 0.1% BW', fontsize=fontsize)
    ax.tick_params(axis='both',labelsize=fontsize)
    ax.set_ylim(1,2e7)
    ax.set_xlim(1,1e5)

    ax.xaxis.set_major_locator(ticker.LogLocator(base=10, numticks=10))
    ax.yaxis.set_major_locator(ticker.LogLocator(base=10, numticks=10))
    locmin = ticker.LogLocator(base=10.0,subs=(0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9),numticks=15)
    ax.xaxis.set_minor_locator(locmin)
    ax.xaxis.set_minor_formatter(ticker.NullFormatter())

    ax.yaxis.set_minor_locator(locmin)
    ax.yaxis.set_minor_formatter(ticker.NullFormatter())

    plt.savefig("synchrotron-singleFile-spectrum.pdf", dpi=100, bbox_inches='tight')
    plt.savefig("synchrotron-singleFile-spectrum.png", dpi=100, bbox_inches='tight')
    
# ###############################################################################
def integrate_array_with_respect_to_array(array1, array2, start_index, end_index):
    """
    Integrate array1 with respect to array2 between two specified indices using Simpson's rule.

    Parameters:
        array1 (array-like): Array of values to integrate.
        array2 (array-like): Array of values with respect to which integration is performed.
        start_index (int): Start index for integration.
        end_index (int): End index for integration.

    Returns:
        float: Integral of array1 with respect to array2 between the specified indices.
    """
    # Slice the arrays between start and end indices
    array1_slice = array1[start_index:end_index+1]
    array2_slice = array2[start_index:end_index+1]

    # Perform numerical integration using Simpson's rule
    integral = simps(array1_slice, array2_slice)

    return integral

# ############################################
def plot_spectrum_regions(Nphotons_Energy, Energy):
    x_initenergy = 124 #eV
    uv_initenergy = 3.1 #eV

    # Calculate all photons
    start_index = 0            # Start index for integration
    end_index = len(Energy)    # End index for integration
    all_photons = integrate_array_with_respect_to_array(Nphotons_Energy, Energy, start_index, end_index)
    #print('ALL photons number: ', (all_photons))
    #print('ALL photons number: ', "{0:.1E}".format(all_photons))

    for i in range(end_index):
        if Energy[i] >= uv_initenergy:
            uv_index = i
            break
    for i in range(end_index):
        if Energy[i] >= x_initenergy:
            x_index = i
            break
    low_photons = integrate_array_with_respect_to_array(Nphotons_Energy, Energy, start_index, uv_index)
    #print('Low energy photons number (', "{0:.1f}".format(Energy[start_index]), '-', "{0:.1f}".format(Energy[uv_index-1]), ' eV): ', "{0:.1E}".format(low_photons))

    uv_photons = integrate_array_with_respect_to_array(Nphotons_Energy, Energy, uv_index, x_index-1)
    #print('UV photons number (', "{0:.1f}".format(Energy[uv_index]), '-', "{0:.1f}".format(Energy[x_index-1]), ' eV): ', "{0:.1E}".format(uv_photons))

    x_photons = integrate_array_with_respect_to_array(Nphotons_Energy, Energy, x_index, end_index)
    #print('X-ray photons number (', "{0:.1f}".format(Energy[x_index]), '-', "{0:.1f}".format(Energy[end_index-1]), ' eV): ', "{0:.1E}".format(x_photons))

    # PI Plot
    fig,ax = plt.subplots(figsize=(5,5))
    #fig.tight_layout(pad=2.) #set the spacing between subplots
    ##################################################
    Photons = np.array([])
    Photons = np.append(Photons, x_photons)
    Photons = np.append(Photons, uv_photons)
    Photons = np.append(Photons, low_photons)

    mylabels = ["X-ray", "UV", "Low energy photons"]
    myexplode = [0.03, 0.03, 0.03]

    ax.pie(Photons, labels = mylabels, explode = myexplode, autopct='%1.1f%%', textprops={'fontsize': 12})
    plt.savefig('synchrotron-singleFile-pichart-spectrum.png', dpi=100, bbox_inches='tight')
    plt.savefig('synchrotron-singleFile-pichart-spectrum.pdf', dpi=100, bbox_inches='tight')


def plotPhotons_theta_phi(Nphotons_Theta, Nphotons_Phi, Theta, Phi):
    fig,axs = plt.subplots(2, 1, figsize=(5,8))
    fig.tight_layout(pad=4.) #set the spacing between subplots

    ax = axs[0]
    ax.plot(Theta,Nphotons_Theta,c='b')

    ax.set_xlabel('Theta [rad]', fontsize=fontsize)
    ax.set_ylabel('Photons / 0.1% BW', fontsize=fontsize)
    ax.tick_params(axis='both',labelsize=fontsize)
    ax.set_ylim(0,1e8)
    ax.set_xlim(1e-4, 3e-3)



    ax = axs[1]
    ax.plot(Phi,Nphotons_Phi,c='b')

    ax.set_xlabel('Phi [rad]', fontsize=fontsize)
    ax.set_ylabel('Photons / 0.1% BW', fontsize=fontsize)
    ax.tick_params(axis='both',labelsize=fontsize)
    ax.set_ylim(0,1e7)

    plt.savefig('synchrotron-singleFile-Theta-and-Phi.png', dpi=100, bbox_inches='tight')
    plt.savefig('synchrotron-singleFile-Theta-and-Phi.pdf', dpi=100, bbox_inches='tight')

In [ ]:
plot_spectrum(Nphotons_Energy, Energy)
#plot_spectrum_regions(Nphotons_Energy, Energy)
#plotPhotons_theta_phi(Nphotons_Theta, Nphotons_Phi, Theta, Phi)

# plot spectrum for all runs in density scan

In [ ]:
import os

def sort_by_last_numeric(lst):
    # Sort based on the numeric part at the end of each string
    lst_sorted = sorted(lst, key=lambda x: float(x.split('-')[-1]))
    return lst_sorted


def list_folders_in_dir(directory):
    # List all folders in the directory
    folders = [folder for folder in os.listdir(directory) if os.path.isdir(os.path.join(directory, folder))]
    sorted_folders = sort_by_last_numeric(folders)
    return sorted_folders

In [ ]:
#scan_dir   = '/home/novac/Desktop/my_projects/qv3d-automation/data/densScan-febe_acc-fixRadius' #densScan-febe_acc' #'/home/novac/Desktop/my_projects/qv3d-automation/data/densScan-febe_driver/'
scan_dir   = simulation_directory#'/home/novac/Desktop/my_projects/qv3d-automation/data/densScan-febe_acc' #densScan-febe_acc' #'/home/novac/Desktop/my_projects/qv3d-automation/data/densScan-febe_driver/'
folders = list_folders_in_dir(scan_dir)
if simulation_is_running == 1:
    del folders[-1]
folders_path = []
for folder in folders:
    path = os.path.join(scan_dir, folder)
    folders_path.append(path)

In [ ]:
from pathlib import Path
import re


criticalEnergys = []
Nphotons_Energys = []
Nphotons_Thetas  = []
Nphotons_Phis    = []
Energys          = []
Thetas           = []
Phis             = []
phases           = []

for path_h5files in folders_path:
    
    #################################################
    # Split by '-' and take the last part of 'density-7e14'
    density_str = path_h5files.split('density-')[1].split('/')[0]
    density = float(density_str)

    print("{:.2e}".format(density), end="\r")  # Output: 7e14

    density*=1e6 #/m3
    wp = np.sqrt(density*e**2/epsilon_0/m_e)
    kp=wp/c
    lamdap = 2*pi/kp
    E_WB = c*m_e*wp/e # wavebreaking field
    E_WB/=1e9 #GeV
    ####################################################

    # ##################
    # Analyze wakefields
    # ##################
    
    # Specify the folder path
    folder_path = Path(path_h5files)

    # Use Path.glob() to list files matching the pattern
    files = list(folder_path.glob('v3d_synchrotron_*.h5'))

    # Sort the filenames based on the numeric part extracted from each filename
    sorted_file = sorted(files)

    synchFile = sorted_file[-1]

    synch = !python {synchrotron_dotpy} {synchFile}
    
    my_dict = eval(synch[0])

    # Convert NumPy arrays
    for key, value in my_dict.items():
        if isinstance(value, np.ndarray):
            my_dict[key] = value.tolist()

    criticalEnergys.append(my_dict['criticalEnergy'])
    Nphotons_Energys.append(my_dict['Nphoton_Energy'])
    Nphotons_Thetas.append(my_dict['Nphoton_Theta'])
    Nphotons_Phis.append(my_dict['Nphoton_Phi'])
    Energys.append(my_dict['Energy'])
    Thetas.append(my_dict['Theta'])
    Phis.append(my_dict['Phi'])
    phases.append(my_dict['phase'])

In [ ]:
arr = np.array([5e14, 6e14, 7e14, 8e14, 9e14, 1e15, 2e15, 3e15, 4e15, 5e15, 6e15, 7e15, 8e15, 9e15, 1e16, 2e16, 3e16, 4e16, 5e16,
               6e16, 7e16, 8e16, 9e16])#, 1e17])
fontsize = 12
fig,ax = plt.subplots(figsize=(8,5))
fig.tight_layout(pad=4.) #set the spacing between subplots

ax.plot(arr, criticalEnergys,c='b') # photon number ~ beta oscillations * number of electrons
plt.xscale('log')
ax.set_xlabel(r'$\mathrm{Plasma \, density} \, [\mathrm{cm}^{-3}]$', fontsize=12)
ax.set_ylabel('Critical energy [eV]', fontsize=12)
ax.tick_params(axis='both',labelsize=fontsize)
ax.set_ylim(0, 300)
ax.set_xlim(5e14,1e17)

#ax.xaxis.set_major_locator(ticker.LogLocator(base=10, numticks=10))
#ax.yaxis.set_major_locator(ticker.LogLocator(base=10, numticks=10))
#locmin = ticker.LogLocator(base=10.0,subs=(0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9),numticks=15)
#ax.xaxis.set_minor_locator(locmin)
#ax.xaxis.set_minor_formatter(ticker.NullFormatter())

#ax.yaxis.set_minor_locator(locmin)
#ax.yaxis.set_minor_formatter(ticker.NullFormatter())

plt.savefig("criticalEnergy-density.pdf", dpi=100, bbox_inches='tight')
plt.savefig("criticalEnergy-density.png", dpi=100, bbox_inches='tight')

In [ ]:
for i in range( len(Nphotons_Energys) ):
    # plot spectrum
    fig,ax = plt.subplots(figsize=(8,5))
    fig.tight_layout(pad=4.) #set the spacing between subplots
    ax.loglog(Energys[i],Nphotons_Energys[i],c='b') # photon number ~ beta oscillations * number of electrons

    ax.set_xlabel('Photon energy [eV]', fontsize=fontsize)
    ax.set_ylabel('Photons / 0.1% BW', fontsize=fontsize)
    ax.tick_params(axis='both',labelsize=fontsize)
    ax.set_ylim(1,1e7)
    ax.set_xlim(1,5e4)
    
    ax.xaxis.set_major_locator(ticker.LogLocator(base=10, numticks=10))
    ax.yaxis.set_major_locator(ticker.LogLocator(base=10, numticks=10))
    locmin = ticker.LogLocator(base=10.0,subs=(0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9),numticks=15)
    ax.xaxis.set_minor_locator(locmin)
    ax.xaxis.set_minor_formatter(ticker.NullFormatter())

    ax.yaxis.set_minor_locator(locmin)
    ax.yaxis.set_minor_formatter(ticker.NullFormatter())

    scientific_notation = "{:.0e}".format(arr[i])
    txt = 'Plasma Density = ' + str(scientific_notation)
    ax.text(.04, .1, txt, transform=ax.transAxes, fontsize = 12, va='top', color='k')
    
    
    pdf_name = "synchrotron-spectrum" + str(scientific_notation) +".pdf"
    png_name = "synchrotron-spectrum" + str(scientific_notation) +".png"
    plt.savefig(pdf_name, dpi=100, bbox_inches='tight')
    plt.savefig(png_name, dpi=100, bbox_inches='tight')


    # pie plot
    Nphotons_Energy = Nphotons_Energys[i]
    Energy = Energys[i]
    
    x_initenergy = 124 #eV
    uv_initenergy = 3.1 #eV

    # Calculate all photons
    start_index = 0            # Start index for integration
    end_index = len(Energy)    # End index for integration
    all_photons = integrate_array_with_respect_to_array(Nphotons_Energy, Energy, start_index, end_index)
    #print('ALL photons number: ', (all_photons))
    #print('ALL photons number: ', "{0:.1E}".format(all_photons))

    for i in range(end_index):
        if Energy[i] >= uv_initenergy:
            uv_index = i
            break
    for i in range(end_index):
        if Energy[i] >= x_initenergy:
            x_index = i
            break
    low_photons = integrate_array_with_respect_to_array(Nphotons_Energy, Energy, start_index, uv_index)
    #print('Low energy photons number (', "{0:.1f}".format(Energy[start_index]), '-', "{0:.1f}".format(Energy[uv_index-1]), ' eV): ', "{0:.1E}".format(low_photons))

    uv_photons = integrate_array_with_respect_to_array(Nphotons_Energy, Energy, uv_index, x_index-1)
    #print('UV photons number (', "{0:.1f}".format(Energy[uv_index]), '-', "{0:.1f}".format(Energy[x_index-1]), ' eV): ', "{0:.1E}".format(uv_photons))

    x_photons = integrate_array_with_respect_to_array(Nphotons_Energy, Energy, x_index, end_index)
    #print('X-ray photons number (', "{0:.1f}".format(Energy[x_index]), '-', "{0:.1f}".format(Energy[end_index-1]), ' eV): ', "{0:.1E}".format(x_photons))

    # PI Plot
    fig,ax = plt.subplots(figsize=(5,5))
    #fig.tight_layout(pad=2.) #set the spacing between subplots
    ##################################################
    Photons = np.array([])
    Photons = np.append(Photons, x_photons)
    Photons = np.append(Photons, uv_photons)
    Photons = np.append(Photons, low_photons)

    mylabels = ["X-ray", "UV", "Low energy photons"]
    myexplode = [0.03, 0.03, 0.03]

    #ax.text(.5, .8, txt, transform=ax.transAxes, fontsize = 12, va='top', color='k')
    plt.title(txt)
    ax.pie(Photons, labels = mylabels, explode = myexplode, autopct='%1.1f%%', textprops={'fontsize': 12})

    pdf_name = "synchrotron-pie" + str(scientific_notation) +".pdf"
    png_name = "synchrotron-pie" + str(scientific_notation) +".png"
    plt.savefig(pdf_name, dpi=100, bbox_inches='tight')
    plt.savefig(png_name, dpi=100, bbox_inches='tight')